# Prompt Template

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("Ch02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch02-Prompt


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [5]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [7]:
prompt = prompt.format(country= "미국")
prompt

'미국의 수도는 어디인가요?'

In [12]:
prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [13]:
chain = prompt | llm
chain

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115ed92d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115ee5050>, root_client=<openai.OpenAI object at 0x10ddbab90>, root_async_client=<openai.AsyncOpenAI object at 0x115ed9490>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [14]:
chain.invoke("미국").content

'미국의 수도는 워싱턴 D.C.입니다.'

In [15]:
template = "{title}의 작가는 누구인가요?"

prompt = PromptTemplate(
    template= template,
    input_variables=["title"],
)

prompt

PromptTemplate(input_variables=['title'], template='{title}의 작가는 누구인가요?')

In [16]:
prompt.format(title= "드래곤볼")

'드래곤볼의 작가는 누구인가요?'

In [17]:
# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # dictionary 형태로 partial_variables를 전달; 이건 값을 할당하지 않으면 알아서 채워진다는 것임.
    },
)

prompt

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '미국'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [19]:
prompt.format(country1= "대한민국")

'대한민국과 미국의 수도는 각각 어디인가요?'

In [20]:
prompt_partial = prompt.partial(country2= "캐나다")
prompt_partial

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')

In [21]:
prompt_partial.format(country1= "브라질")

'브라질과 캐나다의 수도는 각각 어디인가요?'

In [24]:
chain = prompt_partial | llm
chain

PromptTemplate(input_variables=['country1'], partial_variables={'country2': '캐나다'}, template='{country1}과 {country2}의 수도는 각각 어디인가요?')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x115ed92d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x115ee5050>, root_client=<openai.OpenAI object at 0x10ddbab90>, root_async_client=<openai.AsyncOpenAI object at 0x115ed9490>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [25]:
chain.invoke("브라질").content

'브라질의 수도는 브라질리아이고, 캐나다의 수도는 오타와입니다.'

In [26]:
chain.invoke({"country1": "일본", "country2": "중국"}).content

'일본의 수도는 도쿄이고, 중국의 수도는 베이징입니다.'

In [27]:
from datetime import datetime

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

'February 19'

In [28]:
# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")


In [29]:
prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # dictionary 형태로 partial_variables를 전달
    },
)

In [30]:
prompt.format(n=1)

'오늘의 날짜는 February 19 입니다. 오늘이 생일인 유명인 1명을 나열해 주세요. 생년월일을 표기해주세요.'

In [31]:
chain = prompt | llm

In [32]:
chain.invoke("1").content

'오늘 생일인 유명인 중 한 명은 Millie Bobby Brown입니다. 그녀의 생일은 2004년 2월 19일입니다.'

## ChatPromptTemplate

- system = 시스템 설정 메시지
- human = 사용자 입력 메시지
- ai = ai 답변 메시지

In [33]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")
chat_prompt

ChatPromptTemplate(input_variables=['country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?'))])

In [34]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

# 챗 message 를 생성; format method의 경우 원하는 input을 제공했을 때 prompt의 구성이 어떻게 생겼는지를 알아보기 위함. 추론이랑은 전혀 상관이 없음.
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 테디 입니다.'),
 HumanMessage(content='반가워요!'),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?'),
 HumanMessage(content='당신의 이름은 무엇입니까?')]

In [36]:
llm.invoke(messages).content

'제 이름은 테디입니다. 어떻게 도와드릴까요?'

In [37]:
chain = chat_template | llm
chain.invoke({"name": "백두산", "user_input": "당신의 성은 무엇입니까?"}).content

'제 이름은 백두산입니다. 어떻게 도와드릴까요?'

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"), # 아직 뭘 넣어야 할지 모를 때 이렇게 작성할 수 있음 or 그냥 따로 빼고 싶을 때
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.')), MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지의 대화를 {word_count} 단어로 요약합니다.'))])

In [ ]:
# 아래 포맷을 통해 word_count와 conversation을 input에 전달하여 prompt를 완성시키는 것!
formatted_chat_prompt = chat_prompt.format(
    word_count= 5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)


System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [43]:
chain = chat_prompt | llm | StrOutputParser()

In [44]:
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요, 저는 백두산입니다. 만나서 반갑습니다."
            ),
            (
                "ai",
                "만나서 반가워요. 앞으로 잘 부탁드립니다."
            )
        ]
    }
)

'백두산 만나서 반가워요.'

## Few show prompt

In [45]:
from dotenv import load_dotenv

load_dotenv()


True

In [46]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Ch02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
Ch02-Prompt


In [47]:
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import stream_response

# 객체 생성
llm = ChatOpenAI(
    temperature=0,  # 창의성
    model_name="gpt-4o-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 뭐야?"

# 질의
answer = llm.stream(question)
stream_response(answer)

대한민국의 수도는 서울입니다.